### This notebook will take MC hits and process them such that they can be compared to the isaura tracks information

In [36]:
import sys,os,os.path
#sys.path.append("../../")   # cite IC from parent directory
sys.path.append("/gluster/data/next/software/IC_satkill/")
#sys.path.append("/gluster/data/next/software/IC_sophronia/")
#sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
os.environ['ICTDIR']='/gluster/data/next/software/IC_satkill/'

sys.path.append('/gluster/data/next/notebooks/john_books/fom_fitting')
import functions as func

from IC.invisible_cities.dataflow                  import                  dataflow as  fl
from IC.invisible_cities.cities.components import track_blob_info_creator_extractor
from IC.invisible_cities.types  .ic_types          import         types_dict_tracks
from IC.invisible_cities.core                      import           system_of_units as units
from IC.invisible_cities.evm.event_model     import HitCollection
from IC.invisible_cities.evm.event_model     import Hit
from IC.invisible_cities.evm.event_model     import Cluster
from IC.invisible_cities.types.ic_types     import xy
from IC.invisible_cities.reco.paolina_functions import voxelize_hits, drop_end_point_voxels, make_track_graphs, get_track_energy

from typing                import Dict
import time

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['mathtext.fontset'] = 'stix'
rcParams['font.family'] = 'STIXGeneral'
rcParams['figure.figsize'] = [10, 8]
rcParams['font.size'] = 22

import pandas as pd
import numpy  as np
import tables as tb

In [29]:
def hits_from_df_MC (dst : pd.DataFrame, skip_NN : bool = False) -> Dict[int, HitCollection]:
    """
    Function that transforms pandas DataFrame dst to HitCollection
    ------
    Parameters
    ------
    dst : pd.DataFrame
        DataFrame with obligatory columns :
                event, npeak, X, Y, Z,  Q, E
        If time, nsipm, Xrms, Yrms, Qc, Ec, track_id are not
        inside dst the default value is set to -1
        If Xpeak, Ypeak not in dst the default value is -1000
    ------
    Returns
    ------
    Dictionary {event_number : HitCollection}
    """
    all_events = {}
    times = getattr(dst, 'time', [-1]*len(dst))
    for (event, time) , df in dst.groupby(['event_id', times]):
        #pandas is not consistent with numpy dtypes so we have to change it by hand
        event = np.int32(event)
        hits  = []
        for i, row in df.iterrows():
            Q = getattr(row,'Q', row.energy)
            if skip_NN and Q == NN:
                continue
            if hasattr(row, 'Xrms'):
                Xrms  = row.Xrms
                Xrms2 = Xrms**2
            else:
                Xrms = Xrms2 = -1
            if hasattr(row, 'Yrms'):
                Yrms  = row.Yrms
                Yrms2 = Yrms**2
            else:
                Yrms = Yrms2 = -1
            nsipm   = getattr(row, 'nsipm'   , -1   )     # for backwards compatibility
            Qc      = getattr(row, 'Qc'      , -1   )     # for backwards compatibility
            Xpeak   = getattr(row, 'Xpeak'   , -1000)     # for backwards compatibility
            Ypeak   = getattr(row, 'Ypeak'   , -1000)     # for backwards compatibility
            Ec      = getattr(row, 'Ec'      , -1   )     # for backwards compatibility
            trackID = getattr(row, 'track_id', -1   )     # for backwards compatibility
            Ep      = getattr(row, "Ep"      , -1   )     # for backwards compatibility

            hit = Hit(row.npeak            ,
                      Cluster(Q               ,
                              xy(row.x, row.y),
                              xy(Xrms2, Yrms2),
                              nsipm = nsipm   ,
                              z     = row.z   ,
                              E     = row.energy   ,
                              Qc    = Qc      ),
                      row.z                ,
                      row.energy                ,
                      xy(Xpeak, Ypeak)     ,
                      s2_energy_c = Ec     ,
                      track_id    = trackID,
                      Ep          = Ep     )

            hits.append(hit)

        if len(hits):
            all_events[event] = HitCollection(event, time, hits=hits)

    return all_events


def count_tracks_mc(hits_deco):
   
    # stuff needed for paolina track counting
    energy_threshold = 10
    min_voxels = 3
    
    base_vsize = 12 #mm
    the_hits = []

    xs = hits_deco.x
    ys = hits_deco.y
    zs = hits_deco.z
    es = hits_deco.energy

    for x, y, z, e in zip(xs, ys, zs, es):
        if np.isnan(e): continue
        h = Hit(0, Cluster(0, xy(x,y), xy(0,0), 0), z, e*1000, xy(0,0))
        the_hits.append(h)

    voxels = voxelize_hits(the_hits,
                           np.array([base_vsize, base_vsize, base_vsize]), False)

    (mod_voxels, dropped_voxels) = drop_end_point_voxels(voxels, energy_threshold, min_voxels)

    
    tracks = make_track_graphs(mod_voxels)
    tracks = sorted(tracks, key=get_track_energy, reverse = True)
    
    track_no = 0
    for c, t in enumerate(tracks, 0):
        track_no += 1
    
    return track_no    

In [ ]:
folder_path = '/gluster/data/next/files/TOPOLOGY_John/HYPPOS_DATA_QTHR/BEERSHEBA_STUDY/satkill/PORT_1a/isaura/'


# take the event energy from the summary
file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f.endswith('.h5')]


dfs = []

for file in file_names:
    file_path = folder_path + file
    df_MC = pd.read_hdf(file_path, 'MC/hits')
    # do the processing for the hits here
    for pid, df in df_MC.groupby('event_id'):
        # find each track and how much energy should be attributed to it

    dfs.append(df)
    
MC_hit_energy = pd.concat(dfs, axis = 0, ignore_index = True)


In [32]:
# one file first
v_size = 12

paolina_params = dict(
                    vox_size = [v_size * units.mm, v_size * units.mm, v_size * units.mm],
                    strict_vox_size = False,
                    energy_threshold = 10 * units.keV,
                    min_voxels = 3,
                    blob_radius = 18 * units.mm,
                    max_num_hits = 10000)

topology_extractor = track_blob_info_creator_extractor(**paolina_params)

folder_path = '/gluster/data/next/files/TOPOLOGY_John/HYPPOS_DATA_QTHR/BEERSHEBA_STUDY/satkill/PORT_1a/isaura/'
file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f.endswith('.h5')]
df_MC = pd.read_hdf(folder_path + file_names[0], 'MC/hits')
# add npeaks (this is so janky)
df_MC['npeak'] = 1

# generate hit collection from MC (might work?)
mc_hitc = hits_from_df_MC(df_MC)



AttributeError: 'dict' object has no attribute 'hits'

In [33]:
print(mc_hitc)

{80000: HitCollectionHit list:<Hit : npeak = 1 z = 742.98828125 XYpeak = -1000, -1000 E = 0.008066149428486824 Ec = -1 Ep = -1 trackid = -1 cluster =< nsipm = -1 Q = 0.008066149428486824
                    xy = xy(x=405.6971130371094, y=84.1243667602539) 3dHit = Hit(405.6971130371094, 84.1243667602539, 742.98828125, E=0.008066149428486824)  > >, 80001: HitCollectionHit list:<Hit : npeak = 1 z = 989.3580932617188 XYpeak = -1000, -1000 E = 0.0013445025542750955 Ec = -1 Ep = -1 trackid = -1 cluster =< nsipm = -1 Q = 0.0013445025542750955
                    xy = xy(x=138.13656616210938, y=-98.83601379394531) 3dHit = Hit(138.13656616210938, -98.83601379394531, 989.3580932617188, E=0.0013445025542750955)  > >, 80002: HitCollectionHit list:<Hit : npeak = 1 z = 251.62342834472656 XYpeak = -1000, -1000 E = 0.0019958766642957926 Ec = -1 Ep = -1 trackid = -1 cluster =< nsipm = -1 Q = 0.0019958766642957926
                    xy = xy(x=-258.01922607421875, y=335.5096435546875) 3dHit = Hit(-258.0

In [39]:
df_MC_one = df_MC[df_MC.event_id == df_MC.event_id.unique()[0]]
display(df_MC_one)


# stuff needed for paolina track counting
energy_threshold = 10
min_voxels = 3

base_vsize = 12 #mm
the_hits = []

,event_id,particle_id,hit_id,x,y,z,time,energy,label,npeak
0,80000,20,0,457.466187,-42.362900,726.263000,1.617071,0.000234,ACTIVE,1
1,80000,34,0,457.466156,-42.362850,726.263000,1.617098,0.000021,ACTIVE,1
2,80000,33,0,457.466187,-42.362808,726.262817,1.617118,0.000046,ACTIVE,1
3,80000,32,0,457.466278,-42.362698,726.263000,1.617127,0.000045,ACTIVE,1
4,80000,31,0,457.466187,-42.362907,726.263000,1.617076,0.000021,ACTIVE,1
...,...,...,...,...,...,...,...,...,...,...
247,80000,49,11,445.339355,-21.746880,784.309143,2.134957,0.000844,ACTIVE,1
248,80000,49,12,445.342560,-21.754063,784.318542,2.135220,0.001738,ACTIVE,1
249,80000,49,13,445.349548,-21.755783,784.321411,2.135416,0.002368,ACTIVE,1
250,80000,49,14,445.351959,-21.754690,784.322266,2.135517,0.001516,ACTIVE,1


In [133]:
display(df_MC)
display(df_MC[df_MC.label == 'ACTIVE'])

,event_id,particle_id,hit_id,x,y,z,time,energy,label
0,1000000,23,0,-62.902893,312.907623,212.046997,1.887960,0.000185,ACTIVE
1,1000000,34,0,-62.902672,312.907623,212.046997,1.888017,0.000044,ACTIVE
2,1000000,33,0,-62.902832,312.907532,212.046951,1.887995,0.000033,ACTIVE
3,1000000,32,0,-62.902912,312.907593,212.047012,1.887973,0.000021,ACTIVE
4,1000000,31,0,-62.902870,312.907654,212.046997,1.887967,0.000046,ACTIVE
...,...,...,...,...,...,...,...,...,...
123489,1000340,59,1,-78.538589,-206.739258,762.828735,3.137650,0.000382,ACTIVE
123490,1000340,58,0,-78.538071,-206.738983,762.828674,3.137652,0.000101,ACTIVE
123491,1000340,57,0,-78.539032,-206.739105,762.830566,3.137662,0.000094,ACTIVE
123492,1000340,57,1,-78.540787,-206.739182,762.833557,3.137778,0.002058,ACTIVE


,event_id,particle_id,hit_id,x,y,z,time,energy,label
0,1000000,23,0,-62.902893,312.907623,212.046997,1.887960,0.000185,ACTIVE
1,1000000,34,0,-62.902672,312.907623,212.046997,1.888017,0.000044,ACTIVE
2,1000000,33,0,-62.902832,312.907532,212.046951,1.887995,0.000033,ACTIVE
3,1000000,32,0,-62.902912,312.907593,212.047012,1.887973,0.000021,ACTIVE
4,1000000,31,0,-62.902870,312.907654,212.046997,1.887967,0.000046,ACTIVE
...,...,...,...,...,...,...,...,...,...
123489,1000340,59,1,-78.538589,-206.739258,762.828735,3.137650,0.000382,ACTIVE
123490,1000340,58,0,-78.538071,-206.738983,762.828674,3.137652,0.000101,ACTIVE
123491,1000340,57,0,-78.539032,-206.739105,762.830566,3.137662,0.000094,ACTIVE
123492,1000340,57,1,-78.540787,-206.739182,762.833557,3.137778,0.002058,ACTIVE


In [67]:


xs = df_MC_one.x
ys = df_MC_one.y
zs = df_MC_one.z
es = df_MC_one.energy
print("Sanity check, how much energy before:")
print(np.sum(es))

the_hits = []

for x, y, z, e in zip(xs, ys, zs, es):
    if np.isnan(e): continue
    h = Hit(0, Cluster(0, xy(x,y), xy(0,0), 0), z, e*1000, xy(0,0))
    the_hits.append(h)

voxels = voxelize_hits(the_hits,
                        np.array([base_vsize, base_vsize, base_vsize]), False)

print(voxels)

(mod_voxels, dropped_voxels) = drop_end_point_voxels(voxels, energy_threshold, min_voxels)


tracks = make_track_graphs(mod_voxels)

print(tracks)

E = []
for i in range(len(tracks)):
    E.append(get_track_energy(tracks[i]))


print("Energy after:")
print(sum(E))

print("Number of tracks:")
print(len(tracks))



Sanity check, how much energy before:
0.5831881
[Voxel(411.0962036132788, 78.52730019887564, 743.3333129882808, E=29.781300176182413), Voxel(444.4447265625012, -22.219897905987263, 778.5719604492213, E=55.89614950076793), Voxel(455.5609008789087, -44.608164151512355, 719.8408813476537, E=45.37782451370731), Voxel(455.5609008789087, -44.608164151512355, 731.5870971679673, E=11.72928817140928), Voxel(455.5609008789087, -33.414031028749804, 719.8408813476537, E=72.44643123885908), Voxel(455.5609008789087, -33.414031028749804, 731.5870971679673, E=219.48189511022065), Voxel(455.5609008789087, -22.219897905987263, 719.8408813476537, E=44.81337661854923), Voxel(455.5609008789087, -22.219897905987263, 731.5870971679673, E=103.66185763268732)]
(<networkx.classes.graph.Graph object at 0x1490b469b040>, <networkx.classes.graph.Graph object at 0x1490b47fd4f0>, <networkx.classes.graph.Graph object at 0x1490c41d63d0>)
Energy after:
583.1881229623832
Number of tracks:
3


In [95]:
upper_r = 546 # mm
upper_z = 1206 # mm
lower_z = 3.7 # mm
# check if track falls within or outwith the detector 
for c, t in enumerate(tracks, 0):
    print(c)
    print(t)
    v = t.nodes()
    pos = [h.pos for v in t.nodes() for h in v.hits]
    x, y, z = map(np.array, zip(*pos))
    r = np.sqrt(x**2 + y**2)
    if max(r) > upper_r:
        print("Out of range!")
        continue
    elif (max(z) > upper_z) or (min(z) < lower_z):
        print("Out of range!")
        continue
    else:
        print("Safe!\nMin R: {}   Max R: {}\nMin Z: {}   Max Z: {}\n".format(min(r), max(r), min(z), max(z)))
        # do the shenanigans here


0
Graph with 1 nodes and 0 edges
Safe!
Min R: 414.13854706273526   Max R: 414.33795656508335
Min Z: 742.98828125   Max Z: 743.1243286132812

test
1
Graph with 1 nodes and 0 edges
Safe!
Min R: 445.36723399188406   Max R: 445.9737036613156
Min Z: 784.2437133789062   Max Z: 784.445068359375

test
2
Graph with 6 nodes and 11 edges
Safe!
Min R: 452.9812414693991   Max R: 461.7652288359075
Min Z: 713.9677734375   Max Z: 736.9537353515625

test


In [134]:
def create_row(evt, track_ID, track_E, track_no):
    '''
    create list row for filling df
    '''
    return [evt, track_ID, track_E, track_no]

def MC_track_info(df):
    '''
    Pass through MC hits dataframe and this will produce a new df with formatting:
                                        (MeV)                               (MeV)
    event_id   |   track_ID   |   track energy   | numb_of_tracks  | total energy
        0             0               0.57               2                1.11
        0             1               0.54               2                1.11
        1             0               1.65               1                1.65
        .             .                 .                .                  .
    '''
    # obtained from finding the minima and maxima in the DECO/RECO information
    #upper_r = 546 # mm
    #upper_z = 1206 # mm
    #lower_z = 3.7 # mm
    
    data = []
    
    for (event, df_one) in df.groupby('event_id'):
        
        the_hits = []
        
        # keep the hits within the active region
        df_one = df_one[df_one.label == 'ACTIVE']
        
        xs = df_one.x
        ys = df_one.y
        zs = df_one.z
        es = df_one.energy
        
        
    
        for x, y, z, e in zip(xs, ys, zs, es):
            if np.isnan(e): continue
            h = Hit(0, Cluster(0, xy(x,y), xy(0,0), 0), z, e*1000, xy(0,0))
            the_hits.append(h)

        voxels = voxelize_hits(the_hits,
                        np.array([base_vsize, base_vsize, base_vsize]), False)

        
        (mod_voxels, dropped_voxels) = drop_end_point_voxels(voxels, energy_threshold, min_voxels)

        tracks = make_track_graphs(mod_voxels)
        
        track_number = len(tracks)
        
        for c, t in enumerate(tracks, 0):
            # collect information on track
            #pos = [h.pos for v in t.nodes() for h in v.hits]
            #x, y, z = map(np.array, zip(*pos))
            #r = np.sqrt(x**2 + y**2)
            #if max(r) > upper_r:
            #    #print("Out of range!")
            #    continue
            #elif (max(z) > upper_z) or (min(z) < lower_z):
            #    #print("Out of range!")
            #    continue
            #else:
            #    #print("Safe!\nMin R: {}   Max R: {}\nMin Z: {}   Max Z: {}\n".format(min(r), max(r), min(z), max(z)))
                
            #    # append only tracks that fall within the detector
            data.append(create_row(event, c, get_track_energy(t)/1000, track_number))
        
        
        
    return  pd.DataFrame(data, columns = ['event_id', 'track_ID', 'track_energy', 'numb_of_tracks'])
        


In [135]:
display(MC_track_info(df_MC_one))

,event_id,track_ID,track_energy,numb_of_tracks
0,80000,0,0.029781,3
1,80000,1,0.055896,3
2,80000,2,0.497511,3


### sanity check, do any of the MC hits within active go outwith the volume?
the answer is no, lol

In [124]:
print(np.min(df_MC.z.to_numpy()))
print(np.max(df_MC.z.to_numpy()))
R = np.sqrt(df_MC.x.to_numpy()**2 + df_MC.y.to_numpy()**2)
print(np.max(R), np.min(R))

6.581435
1449.5265
499.50836 3.8328204


In [136]:
folder_path = '/gluster/data/next/files/TOPOLOGY_John/HYPPOS_DATA_QTHR/BEERSHEBA_STUDY/satkill/PORT_1a/isaura/'
file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f)) and f.endswith('.h5')]


dfs = []
i = 0
tot = len(file_names)
for file in file_names:
    df_MC = pd.read_hdf(folder_path + file, 'MC/hits')
    
    dfs.append(MC_track_info(df_MC))
    
    i += 1
    
    if (i%5 == 0):
        print("{}/{}".format(i,tot))
    
MC_hit_energy = pd.concat(dfs, axis = 0, ignore_index = True)


5/300
10/300
15/300
20/300
25/300
30/300
35/300
40/300
45/300
50/300
55/300
60/300
65/300
70/300
75/300
80/300
85/300
90/300
95/300
100/300
105/300
110/300
115/300
120/300
125/300
130/300
135/300
140/300
145/300
150/300
155/300
160/300
165/300
170/300
175/300
180/300
185/300
190/300
195/300
200/300
205/300
210/300
215/300
220/300
225/300
230/300
235/300
240/300
245/300
250/300
255/300
260/300
265/300
270/300
275/300
280/300
285/300
290/300
295/300
300/300


In [137]:
MC_hit_energy.to_hdf('MC_topology_info.h5', 'topology')

In [111]:
display(df_MC)
test_var = df_MC[df_MC.particle_id == 23].index
print(test_var)
display(df_MC.drop(test_var))

,event_id,particle_id,hit_id,x,y,z,time,energy,label
0,1000000,23,0,-62.902893,312.907623,212.046997,1.887960,0.000185,ACTIVE
1,1000000,34,0,-62.902672,312.907623,212.046997,1.888017,0.000044,ACTIVE
2,1000000,33,0,-62.902832,312.907532,212.046951,1.887995,0.000033,ACTIVE
3,1000000,32,0,-62.902912,312.907593,212.047012,1.887973,0.000021,ACTIVE
4,1000000,31,0,-62.902870,312.907654,212.046997,1.887967,0.000046,ACTIVE
...,...,...,...,...,...,...,...,...,...
123489,1000340,59,1,-78.538589,-206.739258,762.828735,3.137650,0.000382,ACTIVE
123490,1000340,58,0,-78.538071,-206.738983,762.828674,3.137652,0.000101,ACTIVE
123491,1000340,57,0,-78.539032,-206.739105,762.830566,3.137662,0.000094,ACTIVE
123492,1000340,57,1,-78.540787,-206.739182,762.833557,3.137778,0.002058,ACTIVE


Int64Index([     0,    634,    635,   2366,   4645,   4991,   6307,   6308,
              7558,   7559,
            ...
            120031, 120032, 120033, 120034, 120036, 121091, 121586, 121587,
            122905, 123175],
           dtype='int64', length=2431)


,event_id,particle_id,hit_id,x,y,z,time,energy,label
1,1000000,34,0,-62.902672,312.907623,212.046997,1.888017,0.000044,ACTIVE
2,1000000,33,0,-62.902832,312.907532,212.046951,1.887995,0.000033,ACTIVE
3,1000000,32,0,-62.902912,312.907593,212.047012,1.887973,0.000021,ACTIVE
4,1000000,31,0,-62.902870,312.907654,212.046997,1.887967,0.000046,ACTIVE
5,1000000,30,0,-62.902901,312.907623,212.046997,1.887968,0.000012,ACTIVE
...,...,...,...,...,...,...,...,...,...
123489,1000340,59,1,-78.538589,-206.739258,762.828735,3.137650,0.000382,ACTIVE
123490,1000340,58,0,-78.538071,-206.738983,762.828674,3.137652,0.000101,ACTIVE
123491,1000340,57,0,-78.539032,-206.739105,762.830566,3.137662,0.000094,ACTIVE
123492,1000340,57,1,-78.540787,-206.739182,762.833557,3.137778,0.002058,ACTIVE
